In [56]:
import json
import networkx as nx
import numpy as np

In [57]:
#function to read .json file of the input graph
def readPreferenceGraph(filePath) :
    with open(filePath) as file :
        data = json.load(file)

    G = nx.MultiDiGraph()

    for edge in data["edges"] :
        G.add_edge(edge["source"], edge["target"], opinion=edge["opinion"], questionItem=edge["question item"])
    
    return G

In [58]:
def calculateProbability(item) :
    # preference
    if item == 0 or item == 1 :
        return 1
    # perception of preference
    elif item == 2 or item == 3 :
        return 0.7
    # avoidance
    elif item == 4 or item == 5 :
        return 0
    # perception of avoidance
    else :
        return 0.3

In [59]:
def calculateIndexes(prefGraph, snapshotGraph, n) :
    # Direct Sociometric Indexes
    Sp = [0] * n # incoming edges to a node - elections
    Pp = [0] * n # incoming edges to a node - perceptions of elections
    Sn = [0] * n # incoming edges to a node - rejections
    Pn = [0] * n # incoming edges to a node - perceptions of rejections
    Rp = [0] * n # number of cases - reciprocal elections
    Rn = [0] * n # number of cases - reciprocal rejections
    Os = [0] * n # number of cases - feeling opposition (election & rejection)
    Ep = [0] * n # outcoming edges from a node - elections
    En = [0] * n # outcoming edges from a node - rejections
    Pap = [0] * n # number of cases - have perception of election and get election
    Pan = [0] * n # number of cases - have perception of rejection and get rejection

    def isReciprocal(node1, node2, weight) :
        for u, v, data in prefGraph.edges(node2, data=True) :
            if v == node1 and data["questionItem"] == weight :
                return True
        return False

    
    for u, v, data in prefGraph.edges(data=True) :
        if snapshotGraph.has_edge(u, v) :
            if data["questionItem"] in {0, 1}:
                Sp[v] += 1
                Ep[u] += 1
            elif data["questionItem"] in {2, 3} :
                Pp[v] += 1
            elif data["questionItem"] in {4, 5} :
                Sn[v] += 1
                En[u] += 1
            else :
                Pn[v] += 1 
    

    for u, v, data in prefGraph.edges(data=True) :        
        if snapshotGraph.has_edge(u, v) :
            if data["questionItem"] in {0, 1} :
                if isReciprocal(u, v, data["questionItem"]) :
                    Rp[u] += 1
                elif isReciprocal(u, v, data["questionItem"] + 4) :
                    Os[u] += 1
                else :
                    None
            elif data["questionItem"] in {4, 5} :
                if isReciprocal(u, v, data["questionItem"]) :
                    Rn[u] += 1
                elif isReciprocal(u, v, data["questionItem"] - 4) :
                    Os[u] += 1
                else :
                    None
            elif data["questionItem"] in {2, 3} :
                if isReciprocal(u, v, data["questionItem"] - 2) :
                    Pap[u] += 1
                else :
                    None
            else :
                if isReciprocal(u, v, data["questionItem"] - 2) :
                    Pan[u] += 1
                else :
                    None

                
    #Compound Sociometric Indexes
    #Individual
    popularity = [item / (n - 1) for item in Sp] 
    antipathy = [item / (n - 1) for item in Sn]
    affectiveConnection = [item1 / item2 if item2 != 0 else 0 for item1, item2 in zip(Rp, Sp)]
    sociometricStatus = [(item1 + item2 - item3 - item4) / (n - 1) for item1, item2, item3, item4 in zip(Sp, Pp, Sn, Pn)]
    positiveExpansion = [item/(n - 1) for item in Ep]
    negativeExpansion = [item/(n - 1) for item in En]
    realisticPerception = [(item1 + item2) / (item3 + item4) if (item3+item4) != 0 else 0 for item1, item2, item3, item4 in zip(Pap, Pan, Sp, Sn)]
    
    #Group
    association = sum(Rp) / (n* (n-1))
    dissociation = sum(Rn) / (n * (n-1))
    s = sum(Sp)
    cohesion =  sum(Rp) / s if s != 0 else 0
    socialIntensity = (sum(Sp) + sum(Sn)) / (n - 1)

    return {
        "individual": 
            {
            "popularity": np.average(popularity),
            "antipathy": np.average(antipathy),
            "affective connection": np.average(affectiveConnection),
            "sociometric status": np.average(sociometricStatus),
            "positive expansion": np.average(positiveExpansion),
            "negative expansion": np.average(negativeExpansion),
            "realistic perception": np.average(realisticPerception)
            },
        "group":
            {
                "association": association,
                "dissociation": dissociation,
                "cohesion": cohesion,
                "social intensity": socialIntensity
            }
        }

In [60]:
def createInteractionGraph(preferenceGraph, threshold, maxInteractions, maxTimestamp) :
    n = preferenceGraph.number_of_nodes()
    activitiesCount = 6

    interactionGraph = []
    interactionGraphIntensity = [0] * (maxTimestamp + 1)
    graphSequenceIndexes = []
    degrees = [0] * n
   
        
    def analyzeQuiz() :
        def findProbBetween(node1, node2) :
            edges_node1_to_node2 = [calculateProbability(data["questionItem"]) for u, v, data in preferenceGraph.edges(node1, data=True) if v == node2]
            edges_node2_to_node1 = [calculateProbability(data["questionItem"]) for u, v, data in preferenceGraph.edges(node2, data=True) if v == node1]

            if len(edges_node1_to_node2) == 0 and len(edges_node2_to_node1) == 0 :
                return None
            elif len(edges_node1_to_node2) == 0 :
                return np.average(edges_node2_to_node1)
            elif len(edges_node2_to_node1) == 0 :
                return np.average(edges_node1_to_node2)
            else :
                return np.average([np.average(edges_node1_to_node2), np.average(edges_node2_to_node1)])        
        
        #dictionary {edge : probability of existing in the Preference Graph}
        edgesProbabilities = {}
        
        #examine edges between nodes in Preference Graph
        #calculate probability. How strong is the connection between people?
        for node in range(n) :
            for other in range(node + 1, n) :
                prob = findProbBetween(node, other)
                if prob is not None :
                    edgesProbabilities[(node, other)] = prob

        #find possible peers for a student
        #step 1: find connecting links with prob < threshold
        possiblePeers = [[] for _ in range(n)]       
        for edge, prob in edgesProbabilities.items() :
            if prob < threshold :
                possiblePeers[edge[0]].append(edge[1])
                possiblePeers[edge[1]].append(edge[0])

        #step 2: keep all the other nodes as possible peers to interact with
        possiblePeers = list(map(lambda item : list(set(range(n)) - set(item[1] + [item[0]])), enumerate(possiblePeers)))
        
        return possiblePeers
    

    def addEdgesAtTimestamp() :
        numberOfInteractions = np.random.choice([2,3])
        G = nx.Graph()
        G.add_nodes_from(range(n))

        for node in range(n) :
            #find for a node how many additional interactions could we add with respect to maxInteractions parameter
            possibleInteractions = maxInteractions - degrees[node]
            if possibleInteractions <= 0 :
                continue

            possibleInteractions = min(numberOfInteractions, possibleInteractions)

            #choose randomly from possible peers
            peers = np.random.choice(possiblePeers[node], size=possibleInteractions, replace=False)
            
            #split possible ways of interaction in sublists and choose randomly an activity from each sublist
            weights = [np.random.choice(sublist) for sublist in np.array_split(range(activitiesCount), possibleInteractions)]

            #add new interaction to the graph
            for peer, weight in zip(peers, weights) :
                G.add_edge(node, peer, interaction_index=weight)
                    
        interactionGraph.append(G)


    def calculateIntensity(time) :
        for node in range(n) :
            count, sum = (0, 0)
            for u, v, data in interactionGraph[time].edges(node, data=True) :
                count += 1
                sum += data["interaction_index"]
            interactionGraphIntensity[time] += count*sum
        


    

    possiblePeers = analyzeQuiz()

    #interactions at timetsamp t = 0
    #interactions based on a scale-free network model with m=3
    interactionGraph.append(nx.barabasi_albert_graph(n=preferenceGraph.number_of_nodes(), m=3))
    for u, v in interactionGraph[0].edges() :
        interactionGraph[0][u][v]["interaction_index"] = np.random.choice(range(activitiesCount))
    calculateIntensity(time=0)
    
    for node in range(n) :
        degrees[node] += interactionGraph[0].degree(node) 
    

    #interactions at timestamps t >= 1
    for time in range(1, maxTimestamp+1) :
        addEdgesAtTimestamp()

        for node in range(n) :
            degrees[node] += interactionGraph[time].degree(node) 

        calculateIntensity(time=time)

        if nx.is_empty(interactionGraph[time]) :
            interactionGraph += [nx.Graph() for _ in range(maxTimestamp - time)]
            for i in range(maxTimestamp - time) :
                calculateIntensity(time=i)
            break
    
    
    
    for time in range(maxTimestamp + 1) :
        graphSequenceIndexes.append(calculateIndexes(prefGraph=preferenceGraph, snapshotGraph=interactionGraph[time], n=n))


    return {
        "graph" : interactionGraph,
        "sequence intensity": interactionGraphIntensity,
        "individual": {
            "popularity": np.average([item["individual"]["popularity"] for item in graphSequenceIndexes]),
            "antipathy": np.average([item["individual"]["antipathy"] for item in graphSequenceIndexes]),
            "affective connection": np.average([item["individual"]["affective connection"] for item in graphSequenceIndexes]),
            "sociometric status": np.average([item["individual"]["sociometric status"] for item in graphSequenceIndexes]),
            "positive expansion": np.average([item["individual"]["positive expansion"] for item in graphSequenceIndexes]),
            "negative expansion": np.average([item["individual"]["negative expansion"] for item in graphSequenceIndexes]),
            "realistic perception": np.average([item["individual"]["realistic perception"] for item in graphSequenceIndexes])
        },
        "group": {
            "association": np.average([item["group"]["association"] for item in graphSequenceIndexes]),
            "dissociation": np.average([item["group"]["dissociation"] for item in graphSequenceIndexes]),
            "cohesion": np.average([item["group"]["cohesion"] for item in graphSequenceIndexes]),
            "social intensity": np.average([item["group"]["social intensity"] for item in graphSequenceIndexes]),
        }
    }

In [63]:
path = "./preferenceGraph.json"

options = {
    "threshold" : 0.1,
    "maxInteractions" : 20,
    "maxTimestamp" : 5
}

G = createInteractionGraph(preferenceGraph=readPreferenceGraph(path), **options)

print(f"Graph Sequence Intensity: {G['sequence intensity']}")
print("\nSnapshots added...")
for i in range(6) :
    print(f"at t = {i}: {G['graph'][i]}")

Graph Sequence Intensity: [13703, 4152, 4056, 3578, 2757, 1028]

Snapshots added...
at t = 0: Graph with 100 nodes and 291 edges
at t = 1: Graph with 100 nodes and 190 edges
at t = 2: Graph with 100 nodes and 185 edges
at t = 3: Graph with 100 nodes and 178 edges
at t = 4: Graph with 100 nodes and 153 edges
at t = 5: Graph with 100 nodes and 83 edges


In [62]:
activities = ["to share content, chat in a social network platform",
            "to participate in debate/group presentation & discussion",
            "to play games",
            "to study in group or work on collaborative projects",
            "to be volunteers",
            "to be teammates in a sports team"]

for i,j,k in G["graph"][1].edges(data=True) :
    print(f"({i},{j}) => interaction: {activities[k['interaction_index']]}")

(0,90) => interaction: to participate in debate/group presentation & discussion
(0,54) => interaction: to study in group or work on collaborative projects
(0,81) => interaction: to be teammates in a sports team
(1,27) => interaction: to play games
(1,85) => interaction: to be teammates in a sports team
(1,30) => interaction: to study in group or work on collaborative projects
(1,32) => interaction: to study in group or work on collaborative projects
(1,68) => interaction: to be teammates in a sports team
(1,89) => interaction: to share content, chat in a social network platform
(2,34) => interaction: to play games
(2,54) => interaction: to be volunteers
(2,70) => interaction: to share content, chat in a social network platform
(3,73) => interaction: to share content, chat in a social network platform
(3,33) => interaction: to be volunteers
(3,4) => interaction: to share content, chat in a social network platform
(3,93) => interaction: to be teammates in a sports team
(4,95) => interact